In [ ]:
! pip install avalanche-lib==0.3.1
! pip install micromind

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Device: {device}")

In [ ]:
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss
from avalanche.training.supervised import Naive
from avalanche.training import JointTraining, Cumulative
from avalanche.training.plugins import EvaluationPlugin, ReplayPlugin
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics
from avalanche.logging import InteractiveLogger
from avalanche.benchmarks.classic import SplitMNIST
import matplotlib.pyplot as plt
import numpy as np
from avalanche.models import SimpleMLP # Import Multi-Layer Perceptron with custom parameters
import torch.nn as nn
import utils

In [ ]:
split_mnist = SplitMNIST(n_experiences=5, seed=1222, return_task_id = True)

# recovering the train and test streams
train_stream = split_mnist.train_stream
test_stream = split_mnist.test_stream

## Training and test function

In [ ]:
# Training Function
def train(model, optimizer, criterion, train_loader):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for images, labels, _ in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        prob = nn.functional.softmax(outputs, dim=1)
        _, predicted = prob.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    accuracy = 100.0 * correct / total
    average_loss = train_loss / len(train_loader)

    return accuracy, average_loss

In [ ]:
def test(model, criterion, test_loader):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels, _ in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            prob = nn.functional.softmax(outputs, dim=1)
            _, predicted = prob.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100.0 * correct / total
    average_loss = test_loss / len(test_loader)

    return accuracy, average_loss

In [ ]:
from torch.utils.data import ConcatDataset

def concat_experience(train_stream, test_stream):
    length = len(train_stream)
    for i in range(0,length):
        if i == 0:
            train_loader = train_stream[i].dataset
            test_loader = test_stream[i].dataset
        else:
            train_loader = torch.utils.data.ConcatDataset([train_loader, train_stream[i].dataset])
            test_loader = torch.utils.data.ConcatDataset([test_loader, test_stream[i].dataset])
    return train_loader, test_loader

In [ ]:
from micromind import PhiNet
MLP1_SGD = SimpleMLP(num_classes=10)

learning_rate = 0.01
momentum = 0.9

optimizer = SGD(MLP1_SGD.parameters(), lr=learning_rate, momentum=momentum)
optimizer = Adam(MLP1_SGD.parameters(), lr=0.01, weight_decay=0.01)
criterion = CrossEntropyLoss()

num_epochs = 10

model = PhiNet(input_shape = (1, 28, 28), alpha = 0.5, beta = 1, t_zero = 6,num_layers=4 ,include_top = True, num_classes = 10).to(device)
Adam_opt = Adam(model.parameters(), lr=0.01, weight_decay=0.01)

In [ ]:
train_loader, test_loader = concat_experience(train_stream, test_stream)
print(len(train_loader))
train_loader = torch.utils.data.DataLoader(train_loader, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_loader, batch_size=64, shuffle=False)

In [ ]:
for epoch in range(num_epochs):
    train_acc, train_loss = utils.train(model, optimizer, criterion, train_loader)
    test_acc, test_loss = utils.test(model, criterion, test_loader)

    print(f"Epoch: {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%, Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")

In [ ]:
for experience in split_mnist.test_stream:
  print("Start of task ", experience.task_label)
  print('Classes in this task:', experience.classes_in_this_experience)
  experience_dataloader = torch.utils.data.DataLoader(experience.dataset, batch_size=64, shuffle=False)
  test_acc, test_loss = test(model, criterion, experience_dataloader)
  print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")